# Initialise Client API

Indicate the project used for making query

In [1]:
from google.cloud import bigquery
import pandas as pd   # for parsing query into dataframe
client = bigquery.Client(project="treenyc-384104") # mark the created project id

# Query

Write the query using SQL

In [2]:
QUERY = (   # query in SQL style
    """
    SELECT *
    FROM `bigquery-public-data.new_york_trees.tree_census_1995`
    LIMIT 10
    """
)

Run the main query and get as a pandas dataframe

In [3]:
query = client.query(QUERY) # create query
df = query.to_dataframe()   # convert into pandas dataframe
print(df)

   recordid                 address house_number           street   
0     21111            700 W 123 ST        700.0         W 123 ST  \
1     21116            700 W 123 ST        700.0         W 123 ST   
2      3219  53-054 WASHINGTON SQ W         None  WASHINGTON SQ W   
3     32660            870/860 1 AV         None             1 AV   
4     32661            870/860 1 AV         None             1 AV   
5     32662            870/860 1 AV         None             1 AV   
6     32663            870/860 1 AV         None             1 AV   
7      3570             767 E 58 ST        767.0          E 58 ST   
8      3571             767 E 58 ST        767.0          E 58 ST   
9     32664            870/860 1 AV         None             1 AV   

   zip_original  cb_original    site species  diameter     status  ...   
0             0            0  Across    QUPA        15       Poor  ...  \
1             0            0  Across    QUPA        13       Good  ...   
2             0   